# Анализ надежности заемщиков  

**Описание проекта.** В настоящем исследовании анализируется, влияет ли семейное положение и количество детей клиента на возможность своевременного возврата кредита, используя статистику по финансовой надёжности клиентов банка. В данном контексте также предоставлены рекомендации по разработке скоринговой модели.

**Ход исследования:** 

1. Обзор данных;
2. Предобработка данных;
3. Выявление зависимостей:
* Зависимость между количеством детей и возвратом кредита в срок;
* Зависимость между семейным положением и возвратом кредита в срок;
* Зависимость между уровнем дохода и возвратом кредита в срок.
4. Итоги исследования.

# Откроем базу данных и изучем общую информацию о данных

In [4]:
#Импортируем необходимые библиотеки
import pandas as pd
import os
working_directory = os.getcwd()
print(working_directory)

/Users/danil


In [6]:
path = working_directory + '/data/data.csv'
data = pd.read_csv(path)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Выведем основную информацию о датафрейме с помощью метода `info()`**

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Предобработка данных

## Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [8]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости сотрудник пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [9]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

## Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом могут быть отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [10]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [11]:
data.groupby('income_type')['days_employed'].agg('median').reset_index()

,income_type,days_employed
0,безработный,366413.652744
1,в декрете,3296.759962
2,госслужащий,2689.368353
3,компаньон,1547.382223
4,пенсионер,365213.306266
5,предприниматель,520.848083
6,сотрудник,1574.202821
7,студент,578.751554


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Выведем перечень уникальных значений столбца `children`.**

In [12]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [13]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца children, чтобы убедиться, что артефакты удален**

In [14]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

## Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`**

In [15]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов

In [16]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`

In [17]:
data['total_income'] = data['total_income'].astype(int)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  int64  
 11  purpose           21402 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


## Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру.

In [19]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [20]:
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates()

## Категоризация данных

**На основании диапазонов, указанных ниже, создим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [22]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [23]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца purpose.**

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [25]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [26]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Исследуем данные и найдем зависимости между метриками

**Есть ли зависимость между количеством детей и возвратом кредита в срок?** 

In [27]:
data_grouped_children = data.groupby('children').agg({'debt': ['count', 'sum']})

In [28]:
data_grouped_children['rate'] = data.groupby('children')['debt'].sum() / data.groupby('children')['debt'].count() * 100
data_grouped_children

debt            rate
          count   sum          
children                       
0         14091  1063  7.543822
1          4808   444  9.234609
2          2052   194  9.454191
3           330    27  8.181818
4            41     4  9.756098
5             9     0  0.000000

**Вывод:** проанализировав зависимость между количеством детей в семье и возратом кредита в срок можно выделить несколько особенностей: 
1. Семей с 5 детьими очень мало и у них нет проблем с возвратом кредитов;
2. Семьи без детей имеют наименьший показатель по возвратам кредитов после семей с 5 детьми и наибольший показатель по полученным кредитам;
3. Наибольшее количество просрочек по возвратам кредитов наблюдается у семей с 1-4 детьми. 

Исходя из этого заемщики без детей выплачивают кредиты исправней, чем семьи с детьми. 

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [29]:
data_grouped_family = data.groupby('family_status').agg({'debt': ['count', 'sum']})
data_grouped_family['rate %'] = data.groupby('family_status')['debt'].sum() / data.groupby('family_status')['debt'].count() * 100
data_grouped_family

debt         rate %
                       count  sum          
family_status                              
Не женат / не замужем   2796  273  9.763948
в разводе               1189   84  7.064760
вдовец / вдова           951   63  6.624606
гражданский брак        4134  385  9.313014
женат / замужем        12261  927  7.560558

**Вывод:** наибольшее количество задолженностей по возврату кредитов наблюдается за людьми не состоящими в браке, а также состоящими в гражданском браке. В то время, как женатые/замужние и вдовцы имеют наименьший показатель по задолженностям 

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [30]:
data_grouped_income = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})
data_grouped_income['rate %'] = data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count() * 100
data_grouped_income

debt          rate %
                       count   sum          
total_income_category                       
A                         25     2  8.000000
B                       5014   354  7.060231
C                      15921  1353  8.498210
D                        349    21  6.017192
E                         22     2  9.090909

**Промежуточные выводы:** исходя из полученных данных можно отметить, что количество людей в категориях A, D, E являются небольшим, в связи с этим, при дальнейшем анализе данных категорий на предмет возврата кредитов в срок, у нас могут быть неточные данные. Исключим данные категории.

In [31]:
data = data[(data['total_income_category'] != 'A')&(data['total_income_category'] != 'D')&(data['total_income_category'] != 'E')]
data_grouped_income = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})
data_grouped_income['rate %'] = data.groupby('total_income_category')['debt'].sum() / data.groupby('total_income_category')['debt'].count() * 100
data_grouped_income

debt          rate %
                       count   sum          
total_income_category                       
B                       5014   354  7.060231
C                      15921  1353  8.498210

**Вывод:** при анализе этой зависимости можно выделить следующее: 
1. Три группы заемщиков A, D, E являются малочисленными и нельзя с уверенность провести анализ этих групп и констатиторовать факт о их надежности или ненадежности. 
2. Самой многочисленной категорией является средний класс (С), у которого наблюдается высокий процент по невозвратам кредитных средств в сроки (8.49) 
3. Исходя из имеющихся данных, наиболее надежными заемщиками являются заемщики категории B, имеющие доход от 200 001 рублей до 1 000 000 рублей. 

Таким образом, заемщики с суммарным доходом  от 200 001 и до 1 000 000 рублей являются самыми надежными. 

**Как разные цели кредита влияют на его возврат в срок?**

In [32]:
data_grouped_purpose = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})
data_grouped_purpose['rate %'] = data.groupby('purpose_category')['debt'].sum() / data.groupby('purpose_category')['debt'].count() * 100
data_grouped_purpose.tail()

debt         rate %
                          count  sum          
purpose_category                              
операции с автомобилем     4203  390  9.279086
операции с недвижимостью  10555  771  7.304595
получение образования      3904  366  9.375000
проведение свадьбы         2273  180  7.919050

**Вывод:** заемщики для целей опереций с недвижимостью составляют наибольшую часть из представленных в исходных данных и являются самыми исправными плательщиками, в то время как, заемщики с целью получения кредитных средств для операций с автомобилями имеют наивысший процент по невозвратам кредитов в сроки

## Приведем возможные причины появления пропусков в исходных данных.

**Предположения:** В исходных данных имеются пропуски, дубликаты и с столбцы с разными регистрами, что свидетельствует о человеческом факторе при заполнении данных. 
Второй причиной может выступать технический сбой при выгрузке данных в результате которого произошел неудачный перенос данных в таблице, часть из которых была утеряна.

**Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* после обнаружения отсутствия данных в таблице можно было выполнить несколько действий: 
1. Исключить строки с пропусками, однако это привело бы к некорректному анализу, поскольку исходя из полученной информации, пропусков было более 2000 тысяч. 
2. Оставить все как есть, тогда это не позволило бы нам осуществить анализ в целом. 
3. Заполнить пропуски используя методы mean() или median(). При заполнении пропусков методом mean() в данной ситуации он бы исказил реальные значения при появлении очень высоких или низких значений, так было принято решение исопользовать метод median(), который позволил нам сохранить имеющиеся данные и не исказали новые данные на месте пропусков. 

# Общий вывод

В данной работе был проведен анализ надежности заемщиков при получении кредитов в банковских организациях. 
Вначале работы была изучена общая информация о предоставленных данных, анализ которой показал, что в данных имеются пропуски значений, аномальные значения и дубликаты. Для корректировки базы данных было проведено ряд мероприятий: 
- Заполнение пропусков данных;
- Обработка аномальных значений; 
- Изменение типов данных; 
- Обработка дубликатов; 
- Категоризация данных. 

Данные мероприятия позволили перейти к анализу поставленных гипотез. 

В рамках поставленных задач были рассмотрено несколько видов зависимостей, которые могут повлиять на факторы возврата заемщиком кредита в срок, а именно: количество детей в семье, семейное положение, уровень дохода и цели привлечения кредитных средств.

1. Рассматривая зависимость влияния количества детей в семье на возврат кредитов в срок, стоит отметить, что заемщики без детей являются самыми надежными заемщиками. Процент невозврата кредитов в срок составляет 7,54%, в то время, как семьи с детьми имеют самые худшие показатели по возврату кредитных средств в сроки. Наибольший показетель по невозврату кредита в срок имеют семьи с двумя детьми (9,45%). 
Категории заемщиков с 4 и 5 детьми не отобразили реальной картины, поскольку в предоставленных данных указанные категории в сумме составляют менее одного процента от общего числа заемщиков, что не позволило с точностью оценить их потенциал на возврат кредита в срок. 

2. При анализе зависимости между семейным положением и возвратом кредита в срок было выявлено, что заемщики, находящиеся в браке, составляют самую большую категорию заемщиков и имеют высокие показатели по возврату кредитов в срок (процент невозврата в срок составил 7,56%), когда заемщики несостоящие в браке являются самыми ненадежными клиентами (9,76%). Немногочисленная категории клиентов "вдовцы" являются самыми надежными плательщикам по своим кредитным обязательствам, процент невозврата кредита в срок составляет 6,62%.

3. Влияние дохода на срок возврата кредита в срок показало, что основная масса кредитов приходится на средний класс (С) с доходом от 50 001 рублей и до 200 000 рублей, соответственно, процент невозврата кредитов в срок у данной категории является высоким и составляет 8,49%. Второй категорией заемщиков является класс выше среднего (B) с доходом от 200 001 рублей и до 1 000 000 рублей. Исходя из полученных данных, они являются надежными заемщиками, процент невозвратов составляет 7,06%. 
В связи с немногочисленными группами заемщиков в категории A, D, E (25, 349, 22 соответственно) анализ данных групп не может отобразить реальной картины. 


4. При рассмотрении гипотезы как влияют цели кредита на срок его возврата в срок, было рассмотрено 4 основаных цели: 
- операции с автомобилями;
- операции с недвижимостью;
- получение образования;
- проведение свадьбы.

Так, анализ показал, что самыми надежными заемщиками выступают люди, берущие кредиты на операции с недвижимостью, процент невозврата кредита в срок - 7,25%, в то время, как заемщики, привлекающие кредиты на операции с автомобилями и берущие кредиты на образование, являются самыми ненадежными, процент невозврата кредита в срок составил - 9,34% и 9,25% соответственно. 


Подводя итоги, можно сделать вывод, что самым надежным классом заемщиков являются заемщики состоящие в браке без детей, имеющие уровень дохода от 200 001 до 1 000 000 рублей и привлекающий кредит на операции с недвижимостью. При получении запросов на выдачу кредитов банкам стоит в обязательном порядке обращать внимания на цель кредитов, статус заемщика (в браке или нет), количество детей в семье и уровень доходов. Также, для более точного представления о надежности клиентов, рекомендуется провести анализ количества дней просрочки кредитов и суммы невозврата кредитов в срок. 